In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import requests
from lxml import html
import unicodecsv as csv
import argparse
import datetime
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import numpy as np
import os, sys, httplib2, json, fire, re, string, requests
from collections import OrderedDict, deque
import re, requests, requests.exceptions
from urllib.parse import urlsplit

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
    

def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors.
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
   

def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
   
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
       
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
               
        if search:
            soup = html            
           
           
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

               
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
           
        return res
   
def get_tag_elements(url, tag='h2'):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
   
    response = simple_get(url)

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        names = set()
        for li in html.select(tag):
            for name in li.text.split('\n'):
                if len(name) > 0:
                    names.add(name.strip())
        return list(names)

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))
   
   
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import requests
from lxml import html
import unicodecsv as csv
import argparse
import datetime
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import numpy as np
import os, sys, httplib2, json, fire, re, string, requests
from collections import OrderedDict, deque
import re, requests, requests.exceptions
from urllib.parse import urlsplit


def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def parameters():
    """
    Function to set parameters for search on yellow pages. Set parameter to True to include in search. Must have
    at least one true for each parameter

    """
    keyword_dict = {
        'clothes': True
    }

    city_province_dict = {
        'Toronto+ON': True
    }

    if sum(keyword_dict.values()) == 0 or sum(city_province_dict.values()) == 0:
        sys.exit("Ensure you have specified at least one keyword and city, province")
    else:
        return keyword_dict, city_province_dict

def parse_listing(keyword, place_city_province, pagination=1):
    """

    Function to process yellowpage listing page 
    : param keyword: search query
    : param place_city : place city name
    : param place_province: place province name

    """

    url = "https://www.yellowpages.ca/search/si/{2}/{0}/{1}".format(keyword, place_city_province, pagination)
    # /pagenumber/keyword/placecity+placeprovince

    print("retrieving ",url)

    headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                'Accept-Encoding':'gzip, deflate, br',
                'Accept-Language':'en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7',
                'Cache-Control':'max-age=0',
                'Connection':'keep-alive',
                'Host':'www.yellowpages.ca',
                'Upgrade-Insecure-Requests':'1',
                'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
            }

    soup = BeautifulSoup(simple_get(url), "html.parser")
    elements = soup.select("a.mlr__item__cta")
    elements = [str(e) for e in elements if "data-phone" in str(e)]
    phones = [e.split("data-phone")[-1].split('"')[1] for e in elements]

    # Adding retries
    for retry in range(100):
        try:
            response = requests.get(url,verify=False, headers = headers )

            if response.status_code==200:
                parser = html.fromstring(response.text)

                #making links absolute
                base_url = "https://www.yellowpages.ca"
                parser.make_links_absolute(base_url)

                XPATH_LISTINGS = "//div[contains(@class, 'listing listing--bottomcta')]" #listings
                listings = parser.xpath(XPATH_LISTINGS)
                print(len(listings), len(phones))
                scraped_results = []


                for results in listings:

                    XPATH_BUSINESS_NAME = ".//a[contains(@class, 'listing__name--link')]//text()"
                    XPATH_BUSINESS_PAGE = ".//a[contains(@class, 'listing__name--link')]//@href"
                    XPATH_STREET = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='streetAddress']//text()"
                    XPATH_LOCALITY = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='addressLocality']//text()"
                    XPATH_REGION = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='addressRegion']//text()"
                    XPATH_ZIP_CODE = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='postalCode']//text()"
                    #XPATH_WEBSITE = ".//div[contains(@class, 'listing__mlr__root')]//ul/li[4]//@href"
                    XPATH_WEBSITE = ".//a[contains(@class, 'mlr__item__cta')]//@href"
                    #XPATH_PHONE = ".//a[contains(@class, 'mlr__item__cta jsMLrMenu')]//@href"


                    raw_business_name = results.xpath(XPATH_BUSINESS_NAME)
                    raw_business_page = results.xpath(XPATH_BUSINESS_PAGE)
                    raw_website = results.xpath(XPATH_WEBSITE)
                    raw_street = results.xpath(XPATH_STREET)
                    raw_locality = results.xpath(XPATH_LOCALITY)
                    raw_region = results.xpath(XPATH_REGION)
                    raw_zip_code = results.xpath(XPATH_ZIP_CODE)
                    # raw_phone = results.xpath(XPATH_PHONE)

                    business_name = ''.join(raw_business_name).strip() if raw_business_name else None
                    business_page = ''.join(raw_business_page).strip() if raw_business_page else None
                    street = ''.join(raw_street).strip() if raw_street else None
                    website = ''.join(raw_website).strip() if raw_website else None
                    locality = ''.join(raw_locality).replace(',\xa0','').strip() if raw_locality else None
                    region = ''.join(raw_region).strip() if raw_region else None
                    zipcode = ''.join(raw_zip_code).strip() if raw_zip_code else None
                    # phone = ''.join(raw_phone).strip() if raw_phone else None

                    business_details = {
                                        'business_name':business_name,
                                        'business_page':business_page,
                                        'street': street,
                                        'website':website,
                                        'street':street,
                                        'locality':locality,
                                        'region':region,
                                        'zipcode':zipcode,
                                        'search_category': keyword,
                                        'phone':phones,
                                        'updated_at': datetime.datetime.now()
                    }
                    scraped_results.append(business_details)
                df = pd.DataFrame(scraped_results)
                df["tel"] = phones
                return df

            elif response.status_code==404:
                print("Could not find a location matching",place)
                break
            else:
                print("Failed to process page")
                return []

        except:
            print("Failed to process page")
            return []

In [20]:
df = parse_listing("clothes", "Toronto+ON")
df.sample(5)

retrieving  https://www.yellowpages.ca/search/si/1/clothes/Toronto+ON


/home/ada/anaconda3/envs/upwork/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


35 35


,business_name,business_page,street,website,locality,region,zipcode,search_category,phone,updated_at,tel
19,Ebène Mode,https://www.yellowpages.ca/bus/Quebec/Montreal...,5-4140 Rue De Chambly,javascript:void(0)https://www.yellowpages.ca/m...,Montréal,QC,H1X 3J6,clothes,"[647-498-0081, 647-498-0081, 416-463-1115, 416...",2022-02-03 13:35:45.703495,438-886-5120
31,Heretic Huculiak,https://www.yellowpages.ca/bus/Ontario/Etobico...,2368 Lake Shore Blvd W,javascript:void(0)https://www.yellowpages.ca/m...,Etobicoke,ON,M8V 1B6,clothes,"[647-498-0081, 647-498-0081, 416-463-1115, 416...",2022-02-03 13:35:45.706445,416-364-4806
7,Due West Clothing Co/next door,https://www.yellowpages.ca/bus/Ontario/Toronto...,431 Queen St W,javascript:void(0)https://www.yellowpages.ca/m...,Toronto,ON,M5V 2A5,clothes,"[647-498-0081, 647-498-0081, 416-463-1115, 416...",2022-02-03 13:35:45.699941,416-593-6267
10,Roots,https://www.yellowpages.ca/bus/Ontario/Toronto...,1485 Yonge Street,javascript:void(0)https://www.yellowpages.ca/m...,Toronto,ON,M4T 1Z2,clothes,"[647-498-0081, 647-498-0081, 416-463-1115, 416...",2022-02-03 13:35:45.700784,416-967-4499
15,Roots,https://www.yellowpages.ca/bus/Ontario/Toronto...,3401 Dufferin Street,javascript:void(0)https://www.yellowpages.ca/m...,Toronto,ON,M6A 2T9,clothes,"[647-498-0081, 647-498-0081, 416-463-1115, 416...",2022-02-03 13:35:45.702312,416-783-3371


In [33]:
df["web"] = df.website.str.split("Fwww.")
# df["web"] = df.web.str.split("%").str[0]

df.sample()

,business_name,business_page,street,website,locality,region,zipcode,search_category,phone,updated_at,tel,web
7,Due West Clothing Co/next door,https://www.yellowpages.ca/bus/Ontario/Toronto...,431 Queen St W,javascript:void(0)https://www.yellowpages.ca/m...,Toronto,ON,M5V 2A5,clothes,"[647-498-0081, 647-498-0081, 416-463-1115, 416...",2022-02-03 13:35:45.699941,416-593-6267,[javascript:void(0)https://www.yellowpages.ca/...


In [40]:
df.website.str.split("Fwww.").str[-1]

0                  toms-place.com%2Fjavascript:void(0);
1                  toms-place.com%2Fjavascript:void(0);
2                      korrys.com%2Fjavascript:void(0);
3     javascript:void(0)https://www.yellowpages.ca/m...
4             eriettaboutique.com%2Fjavascript:void(0);
5     gucci.com%2Fca%2Fen%2Fstore%2F130-bloor-street...
6                        facebook.com%2F137272569651286
7     javascript:void(0)https://www.yellowpages.ca/m...
8     javascript:void(0)https://www.yellowpages.ca/m...
9     roots.com%2Fca%2Fen%2Flocations%2FON%2Ftoronto...
10    roots.com%2Fca%2Fen%2Flocations%2FON%2Ftoronto...
11    roots.com%2Fca%2Fen%2Flocations%2FON%2Ftoronto...
12    roots.com%2Fca%2Fen%2Flocations%2FON%2Ftoronto...
13    roots.com%2Fca%2Fen%2Flocations%2FON%2Ftoronto...
14    roots.com%2Fca%2Fen%2Flocations%2FON%2Ftoronto...
15    roots.com%2Fca%2Fen%2Flocations%2FON%2Ftoronto...
16    roots.com%2Fca%2Fen%2Flocations%2FON%2Ftoronto...
17    javascript:void(0)https://www.yellowpages.

In [ ]:
'=http%3A%2F%2Fwww.toms-place.com%2Fjavascript:void(0);'